In [ ]:
# define llm
from llama_index.llms.ollama import Ollama

llm = Ollama(
    model="phi4-mini",
    base_url="http://localhost:11434", 
    request_timeout=360.0,
    temperature=0.01
    )

response = llm.complete("What is the capital of France?")
print(response)

In [ ]:
from pathlib import Path

all_files_gen = Path("./data/").rglob("*")
all_files = [f.resolve() for f in all_files_gen]
all_pdf_files = [f for f in all_files if f.suffix.lower() == ".pdf"]
len(all_pdf_files)


In [ ]:
from llama_index.core import Document
from docling.document_converter import DocumentConverter

converter = DocumentConverter()

In [ ]:
doc = converter.convert(all_pdf_files[0])

In [ ]:
from docling.chunking import HybridChunker

chunker = HybridChunker()
chunk_iter = chunker.chunk(dl_doc=doc.document)

In [ ]:
from transformers import AutoTokenizer

from docling.chunking import HybridChunker,HierarchicalChunker

EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
MAX_TOKENS = 500  # set to a small number for illustrative purposes

tokenizer = AutoTokenizer.from_pretrained(EMBED_MODEL_ID)

chunker = HierarchicalChunker(
    tokenizer=tokenizer,  # instance or model name, defaults to "sentence-transformers/all-MiniLM-L6-v2"
    max_tokens=MAX_TOKENS,  # optional, by default derived from `tokenizer`
    merge_peers=True,  # optional, defaults to True
)
chunk_iter = chunker.chunk(dl_doc=doc.document)
chunks = list(chunk_iter)

In [ ]:
for i, chunk in enumerate(chunks):
    print(f"=== {i} ===")
    txt_tokens = len(tokenizer.tokenize(chunk.text))
    print(f"chunk.text ({txt_tokens} tokens):\n{repr(chunk.text)}")

    ser_txt = chunker.serialize(chunk=chunk)
    ser_tokens = len(tokenizer.tokenize(ser_txt))
    print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")

    print()

In [ ]:
from llama_index.embeddings.ollama import OllamaEmbedding
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

In [ ]:
embed_dim = len(ollama_embedding.get_text_embedding("hi"))


In [ ]:
doc.document.json()